In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from explorer import Explorer

Additionally, Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/setuptools/_distutils/version.py:337: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to 

In [2]:
e = Explorer()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Products loaded.

Keys cleaned.

Attributes loaded.

100%|██████████| 1000/1000 [00:00<00:00, 82244.48it/s]


0 skipped

Loaded 13 goals.


/home/xingkun/webshop/web_agent_site/envs/web_agent_text_env.py:212: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = self._parse_html(html).findAll(text=True)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/gym/envs/registration.py:619: UserWarning: WARN: Env check failed with the following message: You must specify an observation space (cf gym.spaces) cf https://github.com/openai/gym/blob/master/gym/spaces/
You can set `disable_env_checker=True` to disable this check.
  logger.warn(


In [3]:
e.exp_backend._loop_detect_exp_conflict()

[('11-15_14:15:42.985567_9_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]-click[b09kp78g37]-click[x-large]-click[x-large]',
  '11-15_14:15:47.161072_9_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]-click[b09kp78g37]-click[x-large]-click[x-large]-click[x-large]')]